<header>
   <p  style='font-size:36px;font-family:Arial; color:#F0F0F0; background-color: #00233c; padding-left: 20pt; padding-top: 20pt;padding-bottom: 10pt; padding-right: 20pt;'>
       Austin Bike Share
  <br>
       <img id="teradata-logo" src="https://storage.googleapis.com/clearscape_analytics_demo_data/DEMO_Logo/teradata.svg" alt="Teradata" style="width: 125px; height: auto; margin-top: 20pt;">
    </p>
</header>

<p style = 'font-size:20px;font-family:Arial;color:#00233C'><b>Introduction:</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Bike shares are becoming a popular alternative means of transportation. Suppose you had a transportation business servicing the public with various stations where they could access your transportation services. You must ensure you have equipment at the stations when the public needs them. You also know that the weather dramatically impacts the demand for your transportation services. This demonstration shows how to integrate historical trip information with weather information, leveraging Vantage Geospatial and time-series capabilities to improve your service and grow your business.
<br>
The City of Austin makes data available on >649k bike trips over 2013-2017.</p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'><b>Contents:</b></p>
<ol style = 'font-size:16px;font-family:Arial;color:#00233C'>
    <li>Connect to Vantage</li>
    <li>Explore the data</li>
    <li>Create and Explore Temporal, Geospatial and Time index data</li>
    <li>Insights</li>
    <li>Clean up</li>

<hr style='height:2px;border:none;background-color:#00233C;'>
<b style = 'font-size:28px;font-family:Arial;color:#00233C'>1. Connect to Vantage</b>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>You will be prompted to provide the password. Enter your password, press the Enter key, and then use the down arrow to go to the next cell.</p>

In [ ]:
%connect local, hidewarnings=True

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Setup for execution of notebook. Begin running steps with Shift + Enter keys.</p>

In [ ]:
SET query_band='DEMO=AustinBikeShare.ipynb;' update for session;

<hr style='height:1px;border:none;background-color:#00233C;'>
<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>1.1 Getting Data for This Demo</b></p>    
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>We have provided data for this demo on cloud storage. You can either run the demo using foreign tables to access the data without any storage on your environment or download the data to local storage, which may yield faster execution. Still, there could be considerations of available storage. Two statements are in the following cell, and one of them is commented out. You may switch between the modes by changing the comment string.</p>

In [ ]:
-- call get_data('DEMO_AustinBikeShare_cloud');           -- Takes 20 seconds
call get_data('DEMO_AustinBikeShare_local');           -- Takes 50 seconds

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Optional step – if you want to see status of databases/tables created and space used.</p>

In [ ]:
call space_report();

<hr style='height: 2px;border:none;background-color:#00233C;'>
<b style = 'font-size:28px;font-family:Arial;color:#00233C'>2. Explore the data</b>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>As a warm-up, let us look at the tables in our database TRNG_AustinBike.</p>       

In [ ]:
SELECT 
    DatabaseName,
    TableName
FROM
    DBC.Tables
WHERE
    DatabaseName = 'DEMO_AustinBikeShare'

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>We can see that we have three tables in our database. The Trips table contains data on the trips taken using the bikes, the stations table has locations of the bike stations, and the weather table has details about the weather.
    <br>
    <br>
The query below shows the number of rows in each of the tables in the database.</p>

In [ ]:
SELECT
(
    SELECT COUNT(*)
    FROM DEMO_AustinBikeShare.trips
) AS trips,
(
    SELECT COUNT(*)
    FROM DEMO_AustinBikeShare.stations
) AS stations,
(
    SELECT COUNT(*)
    FROM DEMO_AustinBikeShare.weather
) AS weather;

<hr style='height:1px;border:none;background-color:#00233C;'>
<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>2.1 Examine the trips table</b></p>    
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Let's look at the sample data in the trips table.</p>

In [ ]:
SELECT
    *
FROM
    DEMO_AustinBikeShare.trips
SAMPLE 10;

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Which type of subscribers take most of the rides?</p> 

In [ ]:
select top 10 count(trip_id) as ride_count, subscriber_type from DEMO_AustinBikeShare.trips group by subscriber_type order by 1 desc;

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>From the above result we can say that <b>Walk Up</b> rides are <b>250%</b> more than second most popular subscription type.
    <br><br>
    From which station do highest number of trips start?</p>  

In [ ]:
SELECT
    TOP 20
    start_station_name,
    COUNT(trip_id) AS trips
FROM
    DEMO_AustinBikeShare.trips
GROUP BY 1
ORDER BY 2 DESC;

In [ ]:
%chart start_station_name, trips, title = "Trips by station", height = 200, width = 700

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>We see that <b>Riverside @ S. Lamar</b> has the highest number of trips originating from here.</p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Let's see average number of trips originating per from a station.</p>

In [ ]:
SELECT AVG(trips) FROM (
    SELECT
    start_station_name,
    COUNT(1) AS trips
    FROM
        DEMO_AustinBikeShare.trips
    GROUP BY 1
) AS t;

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>We see that the top station <b>Riverside @ S. Lamar</b> has <b>4 times more trips</b> than the average.</p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Now let's look at the pattern of bike usage over time. </p>    

In [ ]:
SELECT
    TRUNC(start_time, 'Month') AS start_Month,
    COUNT(1) AS trips
FROM
    DEMO_AustinBikeShare.trips
GROUP BY 1
ORDER BY 1;

In [ ]:
%chart start_Month, trips, title = "Trips by day", typex = t, width = 700

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>In the above chart we observe few things:</p>
<ol style = 'font-size:16px;font-family:Arial;color:#00233C'>
    <li>There are two months where the data is nearly missing</li>
    <li>The peak usage month is as much as 30k trips in a month</li>
    <li>March and October are first and second busiest months across the data of 4 years.</li>
</ol>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Can this be related to the weather? Is the weather in March and October favorable for biking? Let's see this in the next section.</p>

<hr style='height:1px;border:none;background-color:#00233C;'>
<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>2.2 Examine the weather table</b></p>    

In [ ]:
SELECT * FROM DEMO_AustinBikeShare.weather SAMPLE 10;

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The temperature data is reported hourly (the minutes and seconds are always zero). The temperature columns are in Kelvin, which few people use to decide if it is good bicycle weather, so we will create a view over the weather table to convert the temperature to Fahrenheit. We will also average the temperature for the day.</p>

In [ ]:
REPLACE VIEW austin_weather AS
    SELECT
        TRUNC(dt, 'Month') AS dt, 
        ROUND(AVG((temp - 273.15) * 9/5 + 32) ,0) AS AveTemp,
        SUM(CASE
                WHEN weather_main in ('Rain', 'Mist') THEN 1
                ELSE 0
            END) AS Precip_hours
    FROM DEMO_AustinBikeShare.weather
    GROUP BY 1;

In [ ]:
SELECT * FROM austin_weather ORDER BY 1;

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>If we plot the data, we find we are missing some data, but we get an idea of the typical temperature ranges.  If we look at the hours each month when precipitation occurs, we see some patterns that could also be impacting the number of trips.</p>

In [ ]:
%chart dt, avetemp, width = 800, title = "Average Temperature by Month"

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Here we can observe that for almost all of the March and October months, the temperature is around 70 degrees Fahrenheit. This is a favorable biking temperature as it is neither too cold nor too hot.</p>

In [ ]:
%chart dt, Precip_hours, width = 800, title = "Average Precip Hours by Month"

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>From the above two charts, March and October have favorable conditions for biking, which reflects in the increased bike rides.</p>

<hr style='height:1px;border:none;background-color:#00233C;'>
<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>2.3 Geospatial data</b></p>    

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The Geospatial columns have a type and one or more pairs of Latitude and Longitude. We included the Latitude and Longitude columns in the table so you could see how a simple geospatial feature (a POINT) is represented.
    <br>
For more geospatial datatypes supported by Teradata, please click <a href = 'https://docs.teradata.com/r/Enterprise_IntelliFlex_VMware/Geospatial-Data-Types/Geospatial-Data/Geometry-Types'>here</a>.</p>

In [ ]:
SELECT * FROM DEMO_AustinBikeShare.stations SAMPLE 5;

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Numerous geospatial functions exist, but we can demonstrate the basics by finding the distance from the main office (station_id = 1001) to other stations.</p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>
For more geospatial functions supported by Teradata, please click <a href = 'https://docs.teradata.com/r/Enterprise_IntelliFlex_VMware/Geospatial-Data-Types'>here</a>.</p>

In [ ]:
SELECT
    TOP 10 station.station_id, station.name, 
    ROUND(office.location.ST_SphericalDistance(station.location), 0) Distance_Meters
FROM DEMO_AustinBikeShare.stations station, DEMO_AustinBikeShare.stations office
WHERE office.station_id = 1001
ORDER BY 1;

<hr style='height: 2px;border:none;background-color:#00233C;'>
<b style = 'font-size:28px;font-family:Arial;color:#00233C'>3. Create and Explore Temporal, Geospatial and Time index data</b>
<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>3.1 Create a temporal table with weather data</b></p>    
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Temporal tables store and maintain information concerning time. Using temporal tables, Vantage can process statements and queries that include time-based reasoning. Temporal tables have one or two special columns which store time information:
<ul style='font-size:16px;font-family:Arial;color:#00233C'>
    <li>A transaction-time column records and maintains the period Vantage was aware of the information in the row. Vantage automatically enters and maintains the transaction-time column data and consequently tracks such information's history.</li>
    <li>A valid-time column models the real-world and stores information such as the time an insurance policy or product warranty is valid, the length of employment of an employee, or other information that is important to track and manipulate in a time-aware fashion. When you add a new row to this type of table, you use the valid-time column to specify the time period for which the row information is valid. This is the period of validity (PV) of the information in the row.</li>
</ul>
</p>

In [ ]:
CREATE TABLE weather_temporal (
    begin_dt      TIMESTAMP(6) NOT NULL,
    end_dt        TIMESTAMP(6) NOT NULL,
    temp          FLOAT,
    temp_min      FLOAT,
    temp_max      FLOAT,
    pressure      INTEGER,
    humidity      INTEGER,
    wind_speed    INTEGER,
    wind_deg      INTEGER,
    rain_1h       FLOAT,
    rain_3h       FLOAT,
    clouds        INTEGER,
    weather_id    INTEGER,
    weather_main  VARCHAR(50),
    weather_desc  VARCHAR(50),
    weather_icon  VARCHAR(50),
    PERIOD FOR Weather_Duration(begin_dt,end_dt) AS VALIDTIME
)
PRIMARY INDEX (weather_id);

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Here, we are converting temp, temp_min, and temp_max from Kelvin to Fahrenheit while inserting the data into the weather_temporal table.</p>

In [ ]:
INSERT INTO weather_temporal
SELECT
    dt,
    dt + INTERVAL '59' MINUTE + INTERVAL '59' SECOND,
    round( ((temp - 273.15) * 9/5 + 32 ) ,0),
    round( ((temp_min - 273.15) * 9/5 + 32 ) ,0),
    round( ((temp_max - 273.15) * 9/5 + 32 ) ,0),
    pressure,
    humidity,
    wind_speed,
    wind_deg,
    rain_1h,
    rain_3h,
    clouds,
    weather_id,
    weather_main,
    weather_desc,
    weather_icon
FROM 
    DEMO_AustinBikeShare.weather;

In [ ]:
SEQUENCED VALIDTIME SELECT * FROM weather_temporal SAMPLE 10;

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Now we can efficiently answer time-based reasoning queries faster and efficiently with Temporal tables. For example, was the weather favorable to biking in March and October 2016?</p>

In [ ]:
SELECT
    COUNT(weather_main) AS weather_hours, weather_main
FROM (
    VALIDTIME PERIOD '(2016-03-01, 2016-03-31)'
    SELECT * FROM weather_temporal
) AS dt
GROUP BY weather_main;

In [ ]:
%chart weather_main, weather_hours, width = 500, title = "Duration(in hours) of weather by weather type(for March 2016)"

In [ ]:
SELECT
    COUNT(weather_main) AS weather_hours, weather_main
FROM (
        VALIDTIME PERIOD '(2016-10-01, 2016-10-30)'
        SELECT * FROM weather_temporal
    ) AS dt
GROUP BY weather_main;

In [ ]:
%chart weather_main, weather_hours, width = 500, title = "Duration(in hours) of weather by weather type(for October 2016)"

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The above graphs suggest that March and October 2016 had more days favorable for biking(clear, cloudy, mist), hence explaining the increased number of bike rides.</p>

<hr style='height:1px;border:none;background-color:#00233C;'>
<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>3.2 Create a view for all trips with start/end stations data and a GEOSEQUENCE with start/end lat/long/time</b></p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The code below defines a view which enhances the trip data with a Geosequence field containing the location and time for the start and end points of the trip.</p>

In [ ]:
REPLACE VIEW trips_geo AS
SELECT
    t.bikeid,
    t.trip_ID,
    t.subscriber_type,
    t.start_station_id,
    COALESCE(t.start_station_name, st.NAME) AS start_station_name,
    t.start_time,
    st.status starting_station_status,
    t.end_station_id,
    COALESCE(t.end_station_name, ed.NAME) AS end_station_name,
    t.start_time 
        + CAST(t.duration_minutes/60 AS INTERVAL HOUR(4)) 
        + CAST(t.duration_minutes MOD 60 AS INTERVAL MINUTE(4)) AS end_time,
    ed.status AS End_station_status,
    t.duration_minutes,
    NEW ST_GEOMETRY('ST_POINT' ,st.Longitude, st.Latitude) AS start_location,
    NEW ST_GEOMETRY('ST_POINT' ,ed.Longitude, ed.Latitude) AS end_location,
    CAST('GEOSEQUENCE( ('
        || COALESCE(st.Longitude,-98.272797)
        || ' '
        || COALESCE(st.Latitude,30.578245)
        || ','
        || COALESCE(ed.longitude,-98.272797)
        || ' '
        || COALESCE(ed.latitude,30.578245)
        || '), ('
        || CAST(CAST(t.start_time AS FORMAT 'yyyy-mm-ddbhh:mi:ss') AS VARCHAR(50))
        || ','
        || CAST(CAST(end_time AS FORMAT 'yyyy-mm-ddbhh:mi:ss') AS VARCHAR(50))
        || '), ('
        || '1,2), (0) )' AS ST_GEOMETRY) AS GEOM
FROM
    DEMO_AustinBikeShare.trips AS t
    LEFT JOIN DEMO_AustinBikeShare.stations AS st ON t.start_station_id = st.station_id
    LEFT JOIN DEMO_AustinBikeShare.stations AS ed ON t.end_station_id = ed.station_id;

In [ ]:
SELECT TOP 10 * FROM trips_geo;

<hr style='height:1px;border:none;background-color:#00233C;'>
<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>3.3 Create a Time Index table of the trips to accelerate time related analysis</b></p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Vantage supports tables with a Primary Time Index (PTI), which is used to store and quickly look up data that arrives based on time. This time-aware index distributes data across the units of parallelism. Still, it allows the optimizer to build plans which go directly to the unit of parallelism where the data is stored based on the time constraint.<br><br>
In this case, we will declare the index to have hourly granularity with a baseline time earlier than any date of data we have. Based on the primary index declaration, the database automatically creates the first column with the name TD_TIMECODE. When we insert data, we will use the start_time column as that value.</p>

In [ ]:
CREATE TABLE trips_geo_pti (
    bikeid                    INTEGER,
    trip_id                   BIGINT,
    subscriber_type           VARCHAR(50),
    start_station_id          INTEGER,
    start_station_name        VARCHAR(100),
    starting_station_status   VARCHAR(50),
    end_station_id            INTEGER,
    end_station_name          VARCHAR(100),
    end_time                  TIMESTAMP(6),
    end_station_status        VARCHAR(50),
    duration_minutes          INTEGER,
    geom                      SYSUDTLIB.ST_GEOMETRY(16776192) INLINE LENGTH 9920
)
PRIMARY TIME INDEX (TIMESTAMP(6), DATE '2013-12-20', MINUTES(60));

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>We now populate the local table.  This could take a minute to get data from the cloud storage.</p>

In [ ]:
INSERT INTO trips_geo_pti
SELECT
    start_time,
    bikeid,
    trip_id,
    subscriber_type,
    start_station_id,
    start_station_name,
    starting_station_status,
    end_station_id,
    end_station_name,
    end_time,
    End_station_status,
    duration_minutes,
    geom
FROM
    trips_geo;

In [ ]:
SELECT TOP 10 * FROM trips_geo_pti

<hr style='height:1px;border:none;background-color:#00233C;'>
<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>3.4 Augment trips data with weather data and extract geospatial information</b></p> 
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Finally, we bring the data together with the geosequence trip information with the available weather data, where the weather report period contains the trip's start time (TD_TIMECODE).</p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>
For more geospatial functions supported by Teradata, please click <a href = 'https://docs.teradata.com/r/Geospatial-Data-Types/June-2020'>here</a>.</p>

In [ ]:
CREATE TABLE trips_and_weather AS (
    SELECT 
        t.start_station_name,
        t.end_station_name,
        t.bikeid,
        t.trip_id,
        t.subscriber_type as subscriber_type,
        t.geom.GetInitT() AS pickup_time,
        t.geom.GetFinalT() AS dropoff_time,
        t.geom.ST_POINTN(1).ST_SPHEROIDALDISTANCE(geom.ST_POINTN(2))/1000 AS total_distance,
        t.geom.ST_POINTN(1).ST_X() AS pickup_location_lon,
        t.geom.ST_POINTN(1).ST_Y() AS pickup_location_lat,
        t.geom.ST_POINTN(2).ST_X() AS dropoff_location_lon,
        t.geom.ST_POINTN(2).ST_Y() AS dropoff_location_lat,        
        t.duration_minutes,
        t.TD_TIMECODE as Trip_TIMECODE,
        wt.*
    FROM 
        trips_geo_pti AS t
        INNER JOIN Weather_temporal AS wt ON wt.weather_duration CONTAINS t.TD_TIMECODE
        AND pickup_time >= '2017-07-01 00:00:00'
)
WITH DATA primary index(trip_id);

In [ ]:
SELECT TOP 10 * FROM trips_and_weather WHERE CAST(pickup_time AS DATE) BETWEEN '2017-07-01' AND '2017-07-31'

<hr style='height: 2px;border:none;background-color:#00233C;'>
<b style = 'font-size:28px;font-family:Arial;color:#00233C'>4. Insights</b>
<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>4.1 Average distance traveled w.r.t start stations</b></p>   

In [ ]:
SELECT
    start_station_name, AVG(total_distance), COUNT(trip_id)
FROM trips_and_weather
GROUP BY start_station_name
ORDER BY 2 DESC;

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The above visualization suggests that Main Office has the highest average distance people travel. Even though only ten trips originate from the main station, it still has the highest average distance traveled. These ten trips are very long.</p>

<hr style='height:1px;border:none;background-color:#00233C;'>
<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>4.2 Effect of weather on distance traveled</b></p>   

In [ ]:
SELECT
    TOP 10 SUM(total_distance) AS distance_km, subscriber_type, weather_main
FROM trips_and_weather
GROUP BY subscriber_type, weather_main
ORDER BY 1 DESC;

<hr style='height:1px;border:none;background-color:#00233C;'>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Looking at the results above, walk-up, local365, and local30 subscribers traveled more distance when the weather was clear or cloudy.</p>
<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>4.3 Average trip duration w.r.t subscriber type and trip type</b></p>   

In [ ]:
SELECT
    subscriber_type,
    CASE
        WHEN start_station_name = end_station_name THEN 'Round_Trip'
        ELSE 'Point-to-Point'
    END AS trip_type,
    AVG(duration_minutes) AS time_mins
FROM trips_and_weather
GROUP BY subscriber_type, trip_type
ORDER BY 3 DESC;

<hr style='height:1px;border:none;background-color:#00233C;'>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Looking at the results above, round trips have longer trips than point-to-point for the explorer, walk up and annual members.</p>
<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>4.4 Does the bike require maintenance?</b></p>   

In [ ]:
SELECT
    bikeid, COUNT(*) AS num_trips, sum(total_distance) AS distance,
    CASE
        WHEN distance > 70 THEN 'Recommended'
        ELSE 'Not Required'
    END AS maintenance
FROM trips_and_weather
GROUP BY bikeid
ORDER BY 3 DESC;

In [ ]:
%chart maintenance, bikeid, aggregatey=count, width = 200, title="Maintenance Required?"

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Looking at the results above, 50 bikes require maintenance according to our assumption that we should do bike repairs after every 70 kms.</p>

<hr style='height:2px;border:none;background-color:#00233C;'>
<b style = 'font-size:28px;font-family:Arial;color:#00233C'>5. Clean up</b>
<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>5.1 Work Tables</b></p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Cleanup work tables to prevent errors next time. This section drops all the tables created during the demonstration.</p>

In [ ]:
DROP TABLE weather_temporal;

In [ ]:
DROP TABLE trips_geo_pti;

In [ ]:
DROP TABLE trips_and_weather;

In [ ]:
DROP VIEW trips_geo;

<hr style='height:1px;border:none;background-color:#00233C;'>
<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b> 5.2 Databases and Tables</b></p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The following code will clean up tables and databases created above.</p>

In [ ]:
call remove_data('DEMO_AustinBikeShare')

<p style = 'font-size:16px;font-family:Arial;color:#00233C'><b>Links:</b></p>
<ul style='font-size:16px;font-family:Arial;color:#00233C'>
    <li>Information about Geospatial datatype can be found <a href = 'https://docs.teradata.com/search/all?query=geospatial&content-lang=en-US'>here</a></li>
    <li>Information about Temporal datatype can be found <a href = 'https://docs.teradata.com/search/all?query=temporal&content-lang=en-US'>here</a></li>
</ul>

<footer style="padding-bottom:35px; background:#f9f9f9; border-bottom:3px solid #00233C">
    <div style="float:left;margin-top:14px">ClearScape Analytics™</div>
    <div style="float:right;">
        <div style="float:left; margin-top:14px">
            Copyright © Teradata Corporation - 2023. All Rights Reserved
        </div>
    </div>
</footer>